# 取得檔案

In [ ]:
!wget https://cobis-fs.bme.ncku.edu.tw/DL-lab-exercise/D4-RDDL_practice.zip

--2024-05-09 05:28:14--  https://cobis-fs.bme.ncku.edu.tw/DL-lab-exercise/D4-RDDL_practice.zip
Resolving cobis-fs.bme.ncku.edu.tw (cobis-fs.bme.ncku.edu.tw)... 140.116.175.179
Connecting to cobis-fs.bme.ncku.edu.tw (cobis-fs.bme.ncku.edu.tw)|140.116.175.179|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 586162495 (559M) [application/zip]
Saving to: ‘D4-RDDL_practice.zip’

D4-RDDL_practice.zi 100%[===================>] 559.01M  48.0MB/s    in 12s     

2024-05-09 05:28:27 (45.8 MB/s) - ‘D4-RDDL_practice.zip’ saved [586162495/586162495]



In [ ]:
!unzip /content/D4-RDDL_practice.zip

Archive:  /content/D4-RDDL_practice.zip
   creating: D4-RDDL_practice/
  inflating: __MACOSX/._D4-RDDL_practice  
  inflating: D4-RDDL_practice/.DS_Store  
  inflating: __MACOSX/D4-RDDL_practice/._.DS_Store  
   creating: D4-RDDL_practice/practice_example/
   creating: D4-RDDL_practice/practice_example/RDDL/
  inflating: D4-RDDL_practice/practice_example/.DS_Store  
  inflating: __MACOSX/D4-RDDL_practice/practice_example/._.DS_Store  
   creating: D4-RDDL_practice/practice_example/Input_data/
   creating: D4-RDDL_practice/practice_example/RDDL/CRMI/
  inflating: D4-RDDL_practice/practice_example/RDDL/.DS_Store  
  inflating: __MACOSX/D4-RDDL_practice/practice_example/RDDL/._.DS_Store  
  inflating: D4-RDDL_practice/practice_example/RDDL/requirements.txt  
   creating: D4-RDDL_practice/practice_example/RDDL/training_func/
  inflating: D4-RDDL_practice/practice_example/RDDL/run_train.py  
  inflating: D4-RDDL_practice/practice_example/RDDL/predict.py  
   creating: D4-RDDL_practice/pract

# 新增區段

In [ ]:
%cd /content/D4-RDDL_practice/practice_example/RDDL

/content/D4-RDDL_practice/practice_example/RDDL


In [ ]:
!ls

CRMI	     predict.py		   requirements.txt  run_train.py   training_func
CRMI_pickle  requirements_gpu.txt  run_ensemble.py   split_data.py


In [ ]:
!python run_train.py -name CRMI -m Baseline -nor 0

Model: "custom_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 512, 213)]           0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 512, 213)]           0         []                            
                                                                                                  
 conv1d (Conv1D)             (None, 512, 256)             163840    ['input_1[0][0]',             
                                                                     'input_2[0][0]']             
                                                                                                  
 conv1d_1 (Conv1D)           (None, 512, 128)             98432     ['conv1d[0][0]',   

In [ ]:
import numpy as np

Accuracy = np.mean([0.827, 0.820, 0.827, 0.823, 0.847])
Specificity = np.mean([0.847, 0.847, 0.787, 0.760, 0.853])
Precision = np.mean([0.840, 0.838, 0.802, 0.787, 0.851])
Recall = np.mean([0.807, 0.793, 0.867, 0.887, 0.840])
F1_Score = np.mean([0.823, 0.815, 0.833, 0.834, 0.846])
auROC = np.mean([0.883, 0.881, 0.890, 0.896, 0.904])
auPRC = np.mean([0.873, 0.902, 0.872, 0.894, 0.897])

print("Accuracy:", Accuracy)
print("Specificity:", Specificity)
print("Precision:", Precision)
print("Recall:", Recall)
print("F1_Score:", F1_Score)
print("auROC:", auROC)
print("auPRC:", auPRC)

Accuracy: 0.8288
Specificity: 0.8187999999999999
Precision: 0.8236000000000001
Recall: 0.8388
F1_Score: 0.8301999999999999
auROC: 0.8907999999999999
auPRC: 0.8876


In [ ]:
pos, neg = 750, 750

TP = pos * Recall
FN = pos - TP
TN = neg * Specificity
FP = neg - TN
# print(TP,FN,TN,FP)

FPR = round(FP/(FP+TN),3)
FNR = round(FN/(FN+TP),3)
FDR = round(FP/(TP+FP),3)
FOR = round(FN/(FN+TN),3)
NPV = round(TN/(TN+FN),3)

print(f'FPR:{FPR}\nFNR:{FNR}\nFDR:{FDR}\nFOR:{FOR}\nNPV:{NPV}')

FPR:0.181
FNR:0.161
FDR:0.178
FOR:0.164
NPV:0.836


In [ ]:
from tensorflow.keras.layers import *
from tensorflow.keras import layers
import tensorflow as tf
import keras

from training_func.custom_model import CustomModel
# --------------------------------------------------
def self_defined_model( dropout_rate, model_name=None):

    inputs1 = Input(shape=(512, 213))
    inputs2 = Input(shape=(512, 213))

    # Convolutional layers1

    conv1d = keras.layers.Conv1D(256, 3, strides=1, padding='same')
    x1 = conv1d(inputs1)
    x2 = conv1d(inputs2)

    conv1d_1 = keras.layers.Conv1D(128, 3, strides=1, padding='same')
    x1 = conv1d_1(x1)
    x2 = conv1d_1(x2)

    # Multi-head attention1
    mh1 = layers.MultiHeadAttention(num_heads=4, key_dim=64, value_dim=64)
    x3 = mh1(x1, x2)
    x4 = mh1(x2, x1)
    x1 = layers.Add()([x1, x3])
    x2 = layers.Add()([x2, x4])

    # Convolutional layers2
    xx1=x1
    xx2=x2
    conv1d_2 = keras.layers.Conv1D(128, 1, strides=1, padding='same')
    x1 = conv1d_2(x1)
    x2 = conv1d_2(x2)
    x1 = layers.Add()([xx1, x1])
    x2 = layers.Add()([xx2, x2])

    # Multi-head attention2
    mh2 = layers.MultiHeadAttention(num_heads=4, key_dim=64, value_dim=64)
    x3 = mh2(x1, x2)
    x4 = mh2(x2, x1)
    x1 = layers.Add()([x1, x3])
    x2 = layers.Add()([x2, x4])

    # Convolutional layers3
    conv1d_3 = keras.layers.Conv1D(128, 1, strides=1, padding='same')
    x3 = conv1d_3(x1)
    x4 = conv1d_3(x2)
    x1 = layers.Add()([x1, x3])
    x2 = layers.Add()([x2, x4])

    # Concatenate the two sequences
    x = layers.Add()([x1, x2])
    # Global average pooling
    x = layers.GlobalAveragePooling1D()(x)

    # Dense layers
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(dropout_rate)(x)

    # Softmax layer
    x = layers.Dense(2, activation="softmax")(x)

    model = CustomModel(inputs=[inputs1, inputs2], outputs=x)

    return model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab_Notebooks/D4-RDDL_practice/practice_example/RDDL

/content/drive/MyDrive/Colab_Notebooks/D4-RDDL_practice/practice_example/RDDL


In [ ]:
file_path = '/content/drive/MyDrive/Colab_Notebooks/D4-RDDL_practice'

# 如果檔案存在，刪除它
if os.path.exists(file_path):
    os.remove(file_path)

In [ ]:
import shutil
import os
source_directory = '/content/D4-RDDL_practice'
target_directory = '/content/drive/MyDrive/Colab_Notebooks'
shutil.copytree(source_directory, os.path.join(target_directory, 'D4-RDDL_practice'))

In [ ]:
import shutil
import os
source_directory = '/content/drive/MyDrive/Colab_Notebooks/D4-RDDL_practice'
target_directory = '/content'
shutil.copytree(source_directory, os.path.join(target_directory, 'D4-RDDL_practice'))

'/content/D4-RDDL_practice'